# 3.3 线性回归的简单实现

## 生成数据集

In [1]:
from mxnet import autograd, gluon, np, npx
from d2l import mxnet as d2l

npx.set_np()

true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 读取数据集

In [2]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个Gluon数据迭代器"""
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [3]:
next(iter(data_iter))

[array([[ 0.16916618, -0.55512375],
        [-0.47798827,  1.388212  ],
        [ 1.9380112 , -0.2731147 ],
        [ 1.1763296 , -0.2125029 ],
        [ 0.15074243,  0.5477517 ],
        [ 0.11690255, -0.03865968],
        [-0.6430466 ,  0.65264654],
        [ 0.7995023 ,  1.3597034 ],
        [ 0.5427114 , -0.13436878],
        [ 1.2402922 , -0.47070336]]),
 array([[ 6.4290004 ],
        [-1.4801452 ],
        [ 9.007935  ],
        [ 7.2789474 ],
        [ 2.6464026 ],
        [ 4.562041  ],
        [ 0.69254965],
        [ 1.1824828 ],
        [ 5.723277  ],
        [ 8.289535  ]])]

In [4]:
# nn是神经网络的缩写
from mxnet.gluon import nn

net = nn.Sequential()
net.add(nn.Dense(1))

In [5]:
from mxnet import init

net.initialize(init.Normal(sigma=0.01))

In [6]:
loss = gluon.loss.L2Loss()

In [7]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

In [8]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l.mean().asnumpy():f}')

epoch 1, loss 0.050168
epoch 2, loss 0.000210
epoch 3, loss 0.000052


In [9]:
w = net[0].weight.data()
print(f'w的估计误差： {true_w - w.reshape(true_w.shape)}')
b = net[0].bias.data()
print(f'b的估计误差： {true_b - b}')

w的估计误差： [ 0.00091362 -0.00058508]
b的估计误差： [0.00085783]


In [10]:
loss = gluon.loss.HuberLoss()
for epoch in range(num_epochs):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l.mean().asnumpy():f}')

TypeError: Operator `abs` registered in backend is known as `abs` in Python. This is a legacy operator which can only accept legacy ndarrays, while received an MXNet numpy ndarray. Please call `as_nd_ndarray()` upon the numpy ndarray to convert it to a legacy ndarray, and then feed the converted array to this operator.

## 练习题

练习题 1：如果将小批量的总损失替换为小批量损失的平均值，需要如何更改学习率

学习率除 batch_size

练习题 2：查看深度学习框架文档，它们提供了哪些损失函数和初始化方法？用Huber损失代替原损失，即

$$
\begin{split}l(y,y') = \begin{cases}|y-y'| -\frac{\sigma}{2} & \text{ if } |y-y'| > \sigma \\ \frac{1}{2 \sigma} (y-y')^2 & \text{ 其它情况}\end{cases}\end{split}
$$

练习题 3：如何访问线性回归的梯度